In [1]:
%reload_ext autoreload
%autoreload 2
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.io.wavfile as wv
import scipy.signal as sig
import wave

from datetime import datetime
from keras.models import Sequential, load_model
from keras.layers import Dense

from util import util
from util import WavFileParts
from util.logUtil import LOG, LOG_HEADER
from util.confusionMatrix import ConfusionMatrix


Using TensorFlow backend.


### globale settings

In [2]:
classes = ['music', 'voice', 'environment']

macDir = '/Volumes/SAA_DATA/datasets/'
winDir = 'E:/SAA_DATA/'
osDir = winDir
recordingDir = osDir + '/localizationRecordings'

if osDir == winDir:
    storageFolder = 'E:/SAA_DATA/storedData/'
else:
    storageFolder = '/Users/etto/Desktop/storedData/'

baseSrcDir = osDir + 'localizationFiles/20171025AllExtractionsMic4'
orgWavDirs1 = ['G428_0.0_1.4',
              'G527_0.5_1.4',
              'Studio_2.0_4.2'
              ]

orgWavDirs2 = ['G428_2.1_2.4',
              'G527_1.2_5.8',
              'Studio_3.0_2.0'
              ]

orgsG428 = ['G428_0.0_1.4','G428_2.1_2.4']
orgsG527 = ['G527_0.5_1.4','G527_1.2_5.8']
orgsStudio = ['Studio_2.0_4.2','Studio_3.0_2.0']

NFFT = 1024

chunksBaseDir = 'chunks'
rooms = ['Studio', 'G428', 'G527']

### utility functies

In [3]:
def readSoundChunksDynamic(moduleString):
    chunks = importlib.import_module(moduleString).soundChunks
    wfPts = []
    for jsonString in chunks:
        wfPts.append(WavFileParts.WavFilePartFromJson(jsonString))
    return wfPts

def timeFunction(func):
    """
    Aanroep: bijv. fpc = timeFunction(lambda: getFilesPerCategory(srcDir))
    """
    startTime = datetime.now()
    print('Start: ' + startTime.strftime('%H:%M:%S') + '\n=================')

    res = func()
    
    endTime = datetime.now()
    print('\n=================\nEnd: ' + endTime.strftime('%H:%M:%S'))
    print('Time taken: '),
    print(endTime - startTime)
    print()
    
    return res
    
def storeTestData(allSpectros, allClasses, storageName, keyName):
    filename = storageFolder + storageName + '.hd5'
    df = pd.DataFrame(allSpectros)
    df.to_hdf(path_or_buf=filename, key='spectros_' + keyName)

    df = pd.DataFrame(allClasses)
    df.to_hdf(path_or_buf=filename, key='classes_' + keyName)

def retrieveTestData(storageName, keyName):
    filename = storageFolder + storageName + '.hd5'
    specDf = pd.read_hdf(path_or_buf=filename, key='spectros_' + keyName)
    classesDf = pd.read_hdf(path_or_buf=filename, key='classes_' + keyName)
    return specDf.values, classesDf.values
    

### functies tbv trainen

In [4]:
# Maakt een dictionary aan; per categorie alle files (volledig pad) uit de srcDir
# srcDir is een van de orgWavDirs, bijvoorbeeld
#    localizationFiles/20171025AllExtractionsMic4/G428_0.0_1.4
def getFilesPerCategory(srcDir):
    filesPerCategory = {}
    for catDirLong in glob.glob(srcDir + '/*'):
        catDir = catDirLong.replace('\\', '/')
        catDir = catDir.replace(srcDir + '/', '')

        filesPerCategory[catDir] = []
        for filename in glob.glob(catDirLong + '/*'):
            filename = filename.replace('\\','/')
            filesPerCategory[catDir].append(filename)
    return filesPerCategory

def getFilesPerCatFromMultipleDirs(srcDirs, srcDirsBase=''):
    filesPerCat = {}
    for dirName in srcDirs:
        srcDir = srcDirsBase + '/' + dirName
        fpcNw = getFilesPerCategory(srcDir)
        if not filesPerCat:
            filesPerCat = fpcNw
        else:
            for key in filesPerCat:
                filesPerCat[key] += fpcNw[key]
    return filesPerCat
        

In [5]:
# Maakt een dictionary aan; per categorie de spectrogrammen
def getSpectrosFromFilesPerCategory(filesPerCategory):
    spectros = {}
    for clz in classes:
        spectros[clz] = []
        for filename in filesPerCategory[clz]:
            fs, signal = wv.read(filename)
            freq_array, segment_times, spectrogram = sig.spectrogram(x=signal, fs=fs, nfft=NFFT, noverlap=0)
            spectros[clz].append(spectrogram.T)
    return spectros

In [6]:
def getClassLengths(spectrosPerCat):
    clzLengths = {}
    for clz in classes:
        clzLengths[clz] = sum([np.shape(lst)[0] for lst in spectrosPerCat[clz]])
    return clzLengths

In [7]:
# verwacht invoer van getSpectrosFromFilesPerCategory
# levert traindata op (X_train en Y_train)
def createTrainDataFromSpectros(spectrosPerCat, clzLengths):
    X_train = np.concatenate(spectrosPerCat[classes[0]], axis=0)
    for i in range(1, len(classes)):
        nwSpectros = np.concatenate(spectrosPerCat[classes[i]], axis=0)
        X_train = np.concatenate((X_train,nwSpectros), axis=0)
    
    # one-hot encoding voor Y_train
    nrFiles = clzLengths[classes[0]]
    Y_train = np.array((np.ones(nrFiles),np.zeros(nrFiles), np.zeros(nrFiles))).T

    nrFiles = clzLengths[classes[1]]
    Y_train_nw = np.array((np.zeros(nrFiles), np.ones(nrFiles), np.zeros(nrFiles))).T
    Y_train = np.concatenate((Y_train, Y_train_nw),axis=0)

    nrFiles = clzLengths[classes[2]]
    Y_train_nw = np.array((np.zeros(nrFiles), np.zeros(nrFiles), np.ones(nrFiles))).T
    Y_train = np.concatenate((Y_train, Y_train_nw),axis=0)
    
    return X_train, Y_train

### deep learning model

In [8]:
def create_model(layersizes):
    # create model
    model = Sequential()
    model.add(Dense(layersizes[0], input_dim=513, activation='relu'))
    for lsize in layersizes[1:]:
        model.add(Dense(lsize, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [9]:
def getModelFileName(modelFilePath, baseModelFilename, layers, nrEpochs):
    modelFilename = modelFilePath + baseModelFilename
    for lsize in layers:
        modelFilename = '{}_{}'.format(modelFilename, lsize)
    modelFilename += 'ep{}'.format(nrEpochs)
    modelFilename += '.hd5'
    return modelFilename

In [10]:
def getModelName(baseModelFilename, layers, nrEpochs):
    modelFilename = baseModelFilename
    for lsize in layers:
        modelFilename = '{}_{}'.format(modelFilename, lsize)
    modelFilename += 'ep{}'.format(nrEpochs)
    return modelFilename

In [11]:
def train_model(X_train, Y_train, layers, nrEpochs, modelFilePath, baseModelFilename, batch_size=None):
    soundModel = create_model(layers)
    history = timeFunction(lambda: soundModel.fit(X_train,Y_train, epochs=nrEpochs, shuffle=True, verbose=1, batch_size=batch_size))
    soundModel.save(getModelFileName(modelFilePath, baseModelFilename, layers, nrEpochs))
    plt.plot(history.history['loss'])
    plt.show()

In [12]:
def evaluate_model(X_test, realClasses, layers, nrEpochs, modelFilePath, baseModelFilename):
    soundModel = load_model(getModelFileName(modelFilePath, baseModelFilename, layers, nrEpochs))

    # predicted classes
    predictions = soundModel.predict(X_test)
    predClasses = predictions.argmax(axis=1)

    matrix = ConfusionMatrix(classes)
    for vals in zip(realClasses, predClasses):
        matrix.add(int(vals[0]), int(vals[1]), 1)
    LOG(matrix.toString(),True)
    LOG('', True)
    LOG(matrix.toF1String(), True)
    return matrix

In [13]:
def train_and_evaluate_per_epoch(X_train, Y_train, realClasses, layers, nrEpochs, modelFilePath, baseModelFilename, batch_size=None):
    soundModel = create_model(layers)
    for epNr in range(1, nrEpochs+1):
        LOG('\n*****************\n* Epoch nr {}\n*****************\n'.format(epNr), True)
        soundModel.fit(X_train,Y_train, epochs=1, shuffle=True, verbose=1, batch_size=batch_size)
        soundModel.save(getModelFileName(modelFilePath, baseModelFilename, layers, epNr))
        evaluate_model(X_train, realClasses, layers, epNr, modelFilePath, baseModelFilename)

### functies tbv testen

In [14]:
def createAndStoreTestData(wavFileParts, baseDir, fileDate, micNr, storeFilename, keyName):
    allSpectros = np.array([])
    allClasses = np.array([])

    for wfPt in wavFileParts: #type: WavFilePart
        if not 'Gunshot' in wfPt.getSoundType():
            filename = baseDir + '/{:d}_{:d}_mono{:d}.wav'.format(fileDate, wfPt.fileNr, micNr)
            fs, signal = wv.read(filename)

            classNr = classes.index(wfPt.getSoundType().lower())
            for soundChunk in wfPt.getSoundChunks(micNr):
                startFrame = int(soundChunk[0] * fs)
                endFrame = int(soundChunk[1] * fs)

                sigChunk = signal[startFrame: endFrame]
                freq_array, segment_times, spectrogram = sig.spectrogram(x=sigChunk, fs=fs, nfft=NFFT, noverlap=0)
                if len(allSpectros) == 0:
                    allSpectros = spectrogram.T
                else:
                    allSpectros = np.append(allSpectros, spectrogram.T, axis=0)
                allClasses = np.append(allClasses, classNr * np.ones(len(segment_times)))
                
    storeTestData(allSpectros, allClasses, storeFilename, keyName)


In [15]:
def getTrainDataFromFolders(orgWavDirs):
    fpc = getFilesPerCatFromMultipleDirs(orgWavDirs, baseSrcDir)
    spcs = getSpectrosFromFilesPerCategory(fpc)
    clzLengths = getClassLengths(spcs)
    X_train, Y_train = createTrainDataFromSpectros(spcs, clzLengths)
    # real train classes
    realTrainClasses = np.concatenate((np.zeros(clzLengths[classes[0]]), 
                                  np.ones(clzLengths[classes[1]]), 
                                  2*np.ones(clzLengths[classes[2]])))
    return X_train, Y_train, realTrainClasses


In [16]:
def testModellen(modelFilePath, layerss, micNr, testFileNames, showOverall=True):

    # test data
    keyname = 'mic{}'.format(micNr)

    for layers in layerss:
        for nrEpochs in nrsEpochs:
            overallMatrix = ConfusionMatrix(classes)
            for testFileName in testFileNames:
                testSpecs, testClasses = retrieveTestData(testFileName, keyname)

                LOG_HEADER(getModelName(baseModelFilename, layers, nrEpochs) 
                           + '\n# ' + keyname
                           + '\n# ' + testFileName,
                           True)
                matrix = evaluate_model(testSpecs, testClasses, layers, nrEpochs, modelFilePath, baseModelFilename)
                overallMatrix.addMatrix(matrix.normalizedCopy())
            
            if showOverall:
                overallMatrix = overallMatrix.normalizedCopy()
                LOG_HEADER(getModelName(baseModelFilename, layers, nrEpochs) 
                           + '\n# ' + keyname
                           + '\n# ' + 'testData_Overall',
                           True)
                LOG(overallMatrix.toString(),True)
                LOG('', True)
                LOG(overallMatrix.toF1String(), True)

### trainen van model obv: G428 en G527

In [17]:
baseModelFilename = '20180821_orgG428_G527'
modelFilePath = storageFolder
logPrefix = 'Orgs G428 en G527'
orgWavDirs = orgsG428 + orgsG527
layersList = [[100, 20], [400, 250, 100, 20], [400, 300, 200, 100, 50, 20, 10], 
              [450, 400, 350, 300, 250, 200, 150, 100, 50, 21]]
nrEpochs = 3
nrsEpochs = [1,2,3]

In [18]:
# Train voor alle lagen
X_train, Y_train, realTrainClasses = getTrainDataFromFolders(orgWavDirs)

In [19]:
for layers in layersList:
    LOG_HEADER(logPrefix + ', lagen: {}'.format(str(layers)), True)
    train_and_evaluate_per_epoch(X_train, Y_train, realTrainClasses, 
                                 layers, nrEpochs, modelFilePath, baseModelFilename)

####################################
#
# Orgs G428 en G527, lagen: [100, 20]
#
####################################


*****************
* Epoch nr 1
*****************

Epoch 1/1
754329/754329 [==============================] - 75s 99us/step - loss: 0.1415 - acc: 0.9486
                   m       v       e |  sens   acc
music         232844   10589     213 |  0.96  0.98
voice           5463  235752    3032 |  0.97  0.96
environment      664    7753  258019 |  0.97  0.98
--------------------------------------
prec            0.97    0.93    0.99

F1 overall: 0.96
F1 music: 0.96
F1 voice: 0.95
F1 environment: 0.98


*****************
* Epoch nr 2
*****************

Epoch 1/1
754329/754329 [==============================] - 73s 97us/step - loss: 0.0891 - acc: 0.9671
                   m       v       e |  sens   acc
music         236085    7227     334 |  0.97  0.98
voice           5698  231137    7412 |  0.95  0.97
environment      412    1589  264435 |  0.99  0.99
-----------------------

In [19]:
# Test alle modellen
testModellen(modelFilePath=storageFolder, 
             layerss=layersList, 
             micNr=4, 
             testFileNames=['testData_G428', 'testData_G527', 'testData_Studio'], 
             showOverall=True)                

####################################
#
# 20180821_orgG428_G527_100_20ep1
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          19735     849      55 |  0.96  0.97
voice            688   13480     268 |  0.93  0.96
environment       24     112   13089 |  0.99  0.99
--------------------------------------
prec            0.97    0.93    0.98

F1 overall: 0.96
F1 music: 0.96
F1 voice: 0.93
F1 environment: 0.98

####################################
#
# 20180821_orgG428_G527_100_20ep1
# mic4
# testData_G527
#
####################################

                   m       v       e |  sens   acc
music           8784     126       7 |  0.99  0.98
voice            219    2645       2 |  0.92  0.98
environment        3       1    2487 |  1.00  1.00
--------------------------------------
prec            0.98    0.95    1.00

F1 overall: 0.97
F1 music: 0.98
F1 voice: 0.94
F1 environment: 1.00

############################

                   m       v       e |  sens   acc
music           8784     120      13 |  0.99  0.98
voice            146    2720       0 |  0.95  0.98
environment        0       4    2487 |  1.00  1.00
--------------------------------------
prec            0.98    0.96    0.99

F1 overall: 0.98
F1 music: 0.98
F1 voice: 0.95
F1 environment: 1.00

####################################
#
# 20180821_orgG428_G527_400_250_100_20ep2
# mic4
# testData_Studio
#
####################################

                   m       v       e |  sens   acc
music          45544   14632    3419 |  0.72  0.87
voice            179   36055    1670 |  0.95  0.87
environment       10    2855   39472 |  0.93  0.94
--------------------------------------
prec            1.00    0.67    0.89

F1 overall: 0.84
F1 music: 0.83
F1 voice: 0.79
F1 environment: 0.91

####################################
#
# 20180821_orgG428_G527_400_250_100_20ep2
# mic4
# testData_Overall
#
####################################

       

                   m       v       e |  sens   acc
music          46408   15003    2184 |  0.73  0.88
voice            674   35691    1539 |  0.94  0.87
environment       13    2131   40193 |  0.95  0.96
--------------------------------------
prec            0.99    0.68    0.92

F1 overall: 0.85
F1 music: 0.84
F1 voice: 0.79
F1 environment: 0.93

####################################
#
# 20180821_orgG428_G527_400_300_200_100_50_20_10ep3
# mic4
# testData_Overall
#
####################################

                   m       v       e |  sens   acc
music           0.89    0.10    0.01 |  0.89  0.95
voice           0.04    0.94    0.02 |  0.94  0.94
environment     0.00    0.02    0.98 |  0.98  0.98
--------------------------------------
prec            0.96    0.89    0.97

F1 overall: 0.94
F1 music: 0.92
F1 voice: 0.92
F1 environment: 0.97

####################################
#
# 20180821_orgG428_G527_450_400_350_300_250_200_150_100_50_21ep1
# mic4
# testData_G428
#
##############

### trainen van model obv: G527 en Studio

In [20]:
baseModelFilename = '20180821_orgG527_Studio'
logPrefix = 'Orgs G527 en Studio'
orgWavDirs = orgsG527 + orgsStudio

In [21]:
# Train voor alle lagen
X_train, Y_train, realTrainClasses = getTrainDataFromFolders(orgWavDirs)
for layers in layersList:
    LOG_HEADER(logPrefix + ', lagen: {}'.format(str(layers)), True)
    train_and_evaluate_per_epoch(X_train, Y_train, realTrainClasses, layers, nrEpochs, modelFilePath, baseModelFilename)

####################################
#
# Orgs G527 en Studio, lagen: [100, 20]
#
####################################


*****************
* Epoch nr 1
*****************

Epoch 1/1
732638/732638 [==============================] - 95s 130us/step - loss: 0.2047 - acc: 0.9233
                   m       v       e |  sens   acc
music         231844   12562     789 |  0.95  0.97
voice           9808  225660    9622 |  0.92  0.95
environment      646    5255  236452 |  0.98  0.98
--------------------------------------
prec            0.96    0.93    0.96

F1 overall: 0.95
F1 music: 0.95
F1 voice: 0.92
F1 environment: 0.97


*****************
* Epoch nr 2
*****************

Epoch 1/1
732638/732638 [==============================] - 93s 127us/step - loss: 0.1314 - acc: 0.9509
                   m       v       e |  sens   acc
music         230067   14643     485 |  0.94  0.97
voice           4187  236882    4021 |  0.97  0.96
environment      471    9595  232287 |  0.96  0.98
-------------------

In [22]:
# Test alle modellen
testModellen(modelFilePath=storageFolder, 
             layerss=layersList, 
             micNr=4, 
             testFileNames=['testData_G428', 'testData_G527', 'testData_Studio'], 
             showOverall=True) 

####################################
#
# 20180821_orgG527_Studio_100_20ep1
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          20174     437      28 |  0.98  0.93
voice           2128   12063     245 |  0.84  0.94
environment      980      10   12235 |  0.93  0.97
--------------------------------------
prec            0.87    0.96    0.98

F1 overall: 0.92
F1 music: 0.92
F1 voice: 0.90
F1 environment: 0.95

####################################
#
# 20180821_orgG527_Studio_100_20ep1
# mic4
# testData_G527
#
####################################

                   m       v       e |  sens   acc
music           8866      50       1 |  0.99  0.97
voice            439    2423       4 |  0.85  0.97
environment        8       0    2483 |  1.00  1.00
--------------------------------------
prec            0.95    0.98    1.00

F1 overall: 0.96
F1 music: 0.97
F1 voice: 0.91
F1 environment: 1.00

########################

                   m       v       e |  sens   acc
music           8850      59       8 |  0.99  0.97
voice            364    2502       0 |  0.87  0.97
environment        1       0    2490 |  1.00  1.00
--------------------------------------
prec            0.96    0.98    1.00

F1 overall: 0.97
F1 music: 0.98
F1 voice: 0.92
F1 environment: 1.00

####################################
#
# 20180821_orgG527_Studio_400_250_100_20ep2
# mic4
# testData_Studio
#
####################################

                   m       v       e |  sens   acc
music          57245    5657     693 |  0.90  0.94
voice           2488   34865     551 |  0.92  0.94
environment       68     553   41716 |  0.99  0.99
--------------------------------------
prec            0.96    0.85    0.97

F1 overall: 0.93
F1 music: 0.93
F1 voice: 0.88
F1 environment: 0.98

####################################
#
# 20180821_orgG527_Studio_400_250_100_20ep2
# mic4
# testData_Overall
#
####################################

   

                   m       v       e |  sens   acc
music          57888    5119     588 |  0.91  0.95
voice           1243   36152     509 |  0.95  0.95
environment       51     438   41848 |  0.99  0.99
--------------------------------------
prec            0.98    0.87    0.97

F1 overall: 0.94
F1 music: 0.94
F1 voice: 0.91
F1 environment: 0.98

####################################
#
# 20180821_orgG527_Studio_400_300_200_100_50_20_10ep3
# mic4
# testData_Overall
#
####################################

                   m       v       e |  sens   acc
music           0.96    0.04    0.00 |  0.96  0.95
voice           0.07    0.92    0.01 |  0.92  0.96
environment     0.04    0.01    0.95 |  0.95  0.98
--------------------------------------
prec            0.89    0.95    0.99

F1 overall: 0.94
F1 music: 0.92
F1 voice: 0.94
F1 environment: 0.97

####################################
#
# 20180821_orgG527_Studio_450_400_350_300_250_200_150_100_50_21ep1
# mic4
# testData_G428
#
##########

### trainen van model obv: Studio

In [23]:
baseModelFilename = '20180821_orgStudio_G428'
modelFilePath = storageFolder
logPrefix = 'Orgs Studio en G428'
orgWavDirs = orgsStudio + orgsG428

In [24]:
# Train voor alle lagen
X_train, Y_train, realTrainClasses = getTrainDataFromFolders(orgWavDirs)
for layers in layersList:
    LOG_HEADER(logPrefix + ', lagen: {}'.format(str(layers)), True)
    train_and_evaluate_per_epoch(X_train, Y_train, realTrainClasses, layers, nrEpochs, modelFilePath, baseModelFilename)

####################################
#
# Orgs Studio en G428, lagen: [100, 20]
#
####################################


*****************
* Epoch nr 1
*****************

Epoch 1/1
729643/729643 [==============================] - 119s 162us/step - loss: 0.2157 - acc: 0.9191
                   m       v       e |  sens   acc
music         224260   19008     963 |  0.92  0.96
voice           6522  229505    8082 |  0.94  0.95
environment      517    5954  234832 |  0.97  0.98
--------------------------------------
prec            0.97    0.90    0.96

F1 overall: 0.94
F1 music: 0.94
F1 voice: 0.92
F1 environment: 0.97


*****************
* Epoch nr 2
*****************

Epoch 1/1
729643/729643 [==============================] - 113s 155us/step - loss: 0.1373 - acc: 0.9482
                   m       v       e |  sens   acc
music         229808   14011     412 |  0.94  0.97
voice           6593  229900    7616 |  0.94  0.96
environment      675    3606  237022 |  0.98  0.98
-----------------

In [25]:
# Test alle modellen
testModellen(modelFilePath=storageFolder, 
             layerss=layersList, 
             micNr=4, 
             testFileNames=['testData_G428', 'testData_G527', 'testData_Studio'], 
             showOverall=True) 

####################################
#
# 20180821_orgStudio_G428_100_20ep1
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          19931     662      46 |  0.97  0.97
voice            903   13259     274 |  0.92  0.96
environment       22      24   13179 |  1.00  0.99
--------------------------------------
prec            0.96    0.95    0.98

F1 overall: 0.96
F1 music: 0.96
F1 voice: 0.93
F1 environment: 0.99

####################################
#
# 20180821_orgStudio_G428_100_20ep1
# mic4
# testData_G527
#
####################################

                   m       v       e |  sens   acc
music           8804     105       8 |  0.99  0.98
voice            194    2668       4 |  0.93  0.98
environment        9       0    2482 |  1.00  1.00
--------------------------------------
prec            0.98    0.96    1.00

F1 overall: 0.97
F1 music: 0.98
F1 voice: 0.95
F1 environment: 1.00

########################

                   m       v       e |  sens   acc
music           8811      98       8 |  0.99  0.98
voice            143    2719       4 |  0.95  0.98
environment        8       1    2482 |  1.00  1.00
--------------------------------------
prec            0.98    0.96    1.00

F1 overall: 0.98
F1 music: 0.99
F1 voice: 0.96
F1 environment: 1.00

####################################
#
# 20180821_orgStudio_G428_400_250_100_20ep2
# mic4
# testData_Studio
#
####################################

                   m       v       e |  sens   acc
music          57112    5770     713 |  0.90  0.95
voice            528   36817     559 |  0.97  0.95
environment       51     431   41855 |  0.99  0.99
--------------------------------------
prec            0.99    0.86    0.97

F1 overall: 0.94
F1 music: 0.94
F1 voice: 0.91
F1 environment: 0.98

####################################
#
# 20180821_orgStudio_G428_400_250_100_20ep2
# mic4
# testData_Overall
#
####################################

   

                   m       v       e |  sens   acc
music          59198    3733     664 |  0.93  0.94
voice           3533   33606     765 |  0.89  0.94
environment       81     327   41929 |  0.99  0.99
--------------------------------------
prec            0.94    0.89    0.97

F1 overall: 0.93
F1 music: 0.94
F1 voice: 0.89
F1 environment: 0.98

####################################
#
# 20180821_orgStudio_G428_400_300_200_100_50_20_10ep3
# mic4
# testData_Overall
#
####################################

                   m       v       e |  sens   acc
music           0.97    0.03    0.00 |  0.97  0.93
voice           0.15    0.84    0.01 |  0.84  0.94
environment     0.02    0.00    0.98 |  0.98  0.99
--------------------------------------
prec            0.85    0.97    0.98

F1 overall: 0.93
F1 music: 0.91
F1 voice: 0.90
F1 environment: 0.98

####################################
#
# 20180821_orgStudio_G428_450_400_350_300_250_200_150_100_50_21ep1
# mic4
# testData_G428
#
##########